In [2]:
pip install git+https://github.com/forrestbao/pyeeg.git

  Cloning https://github.com/forrestbao/pyeeg.git to c:\users\faizan\appdata\local\temp\pip-req-build-k3x1tf5l
  Stored in directory: C:\Users\faizan\AppData\Local\Temp\pip-ephem-wheel-cache-yttppi6f\wheels\2d\3f\ad\106d4fc80b61d1ea1fc18e76e7439fd98aa043d83d58eae741
Successfully built pyeeg
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pyeeg as pe
import pickle as pickle
import pandas as pd
import math

from sklearn import svm
from sklearn.preprocessing import normalize

import os
#import tensorflow as tf
import time

In [4]:
channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] #14 Channels chosen to fit Emotiv Epoch+
band = [4,8,12,16,25,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz
subjectList = ['01','02','03']
#List of subjects

In [17]:
def FFT_Processing (sub, channel, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject
                list channel indice
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    meta = []
    with open('C:/Users/faizan/Downloads/data_preprocessed_python/data_preprocessed_python/s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1

        for i in range (0,40):
            # loop over 0-39 trails
            data = subject["data"][i]
            labels = subject["labels"][i]
            start = 0;

            while start + window_size < data.shape[1]:
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y = pe.bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    meta_data = meta_data + list(Y[0])

                meta_array.append(np.array(meta_data))
                meta_array.append(labels)

                meta.append(np.array(meta_array))    
                start = start + step_size
                
        meta = np.array(meta)
        np.save('C:/Users/faizan/Downloads/data_preprocessed_python/data_preprocessed_python/s' + sub, meta, allow_pickle=True, fix_imports=True)


In [18]:
for subjects in subjectList:
    FFT_Processing (subjects, channel, band, window_size, step_size, sample_rate)

In [22]:
data_training = []
label_training = []
data_testing = []
label_testing = []

for subjects in subjectList:

    with open('C:/Users/faizan/Downloads/data_preprocessed_python/data_preprocessed_python/s' + subjects + '.npy', 'rb') as file:
        sub = np.load(file)
        for i in range (0,sub.shape[0]):
            if i % 8 == 0:
                data_testing.append(sub[i][0])
                label_testing.append(sub[i][1])
            else:
                data_training.append(sub[i][0])
                label_training.append(sub[i][1])
           
np.save('C:/Users/faizan/Downloads/data_preprocessed_python/data_preprocessed_python/data_training', np.array(data_training), allow_pickle=True, fix_imports=True)
np.save('C:/Users/faizan/Downloads/data_preprocessed_python/data_preprocessed_python/label_training', np.array(label_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(data_training).shape, np.array(label_training).shape)

np.save('C:/Users/faizan/Downloads/data_preprocessed_python/data_preprocessed_python/data_testing', np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save('C:/Users/faizan/Downloads/data_preprocessed_python/data_preprocessed_python/label_testing', np.array(label_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)



training dataset: (51240, 70) (51240, 4)
testing dataset: (7320, 70) (7320, 4)


In [32]:
with open('C:/Users/faizan/Downloads/data_preprocessed_python/data_preprocessed_python/data_training.npy', 'rb') as fileTrain:
    X  = np.load(fileTrain)
    
with open('C:/Users/faizan/Downloads/data_preprocessed_python/data_preprocessed_python/label_training.npy', 'rb') as fileTrainL:
    Y  = np.load(fileTrainL)
    
X = normalize(X)
Z = np.ravel(Y[:, [1]])

Arousal_Train = np.ravel(Y[:, [0]])
Valence_Train = np.ravel(Y[:, [1]])
Domain_Train = np.ravel(Y[:, [2]])
Like_Train = np.ravel(Y[:, [3]])

In [33]:
import pandas as pd
import keras.backend as K
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical 
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import Model
import timeit
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution1D, MaxPooling1D, ZeroPadding1D
from keras.optimizers import SGD
#import cv2, numpy as np
import warnings
warnings.filterwarnings('ignore')


In [50]:
from keras.utils import to_categorical
y_train = to_categorical(Z)
y_train

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [51]:
x_train = np.array(X[:])

In [52]:
with open('C:/Users/faizan/Downloads/data_preprocessed_python/data_preprocessed_python/data_testing.npy', 'rb') as fileTrain:
    M  = np.load(fileTrain)
    
with open('C:/Users/faizan/Downloads/data_preprocessed_python/data_preprocessed_python/label_testing.npy', 'rb') as fileTrainL:
    N  = np.load(fileTrainL)

M = normalize(M)
L = np.ravel(N[:, [1]])

Arousal_Test = np.ravel(N[:, [0]])
Valence_Test = np.ravel(N[:, [1]])
Domain_Test = np.ravel(N[:, [2]])
Like_Test = np.ravel(N[:, [3]])

In [53]:
x_test = np.array(M[:])

In [54]:
from keras.utils import to_categorical
y_test = to_categorical(L)
y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [55]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [56]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1], 1)

In [57]:
x_train.shape

(51240, 70, 1)

In [71]:
batch_size = 256
num_classes = 10
epochs = 200
input_shape=(x_train.shape[1], 1)

In [72]:
print(input_shape)

(70, 1)


In [73]:
from keras.layers import Convolution1D, ZeroPadding1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense
from keras.regularizers import l2

In [74]:
model = Sequential()
intput_shape=(x_train.shape[1], 1)
model.add(Conv1D(128, kernel_size=3,padding = 'same',activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2)))
model.add(Conv1D(128,kernel_size=3,padding = 'same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2)))
#model.add(Conv1D(64,kernel_size=3,padding = 'same', activation='relu'))
#model.add(MaxPooling1D(pool_size=(2)))
model.add(Flatten())
model.add(Dense(64, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 70, 128)           512       
_________________________________________________________________
batch_normalization_7 (Batch (None, 70, 128)           512       
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 35, 128)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 35, 128)           49280     
_________________________________________________________________
batch_normalization_8 (Batch (None, 35, 128)           512       
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 17, 128)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 2176)             

In [75]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [76]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)

Epoch 1/200
51240/51240 [==============================] - 69s 1ms/step - loss: 2.0102 - accuracy: 0.2650
Epoch 2/200
51240/51240 [==============================] - 72s 1ms/step - loss: 1.7687 - accuracy: 0.3254
Epoch 3/200
51240/51240 [==============================] - 67s 1ms/step - loss: 1.6625 - accuracy: 0.3671
Epoch 4/200
51240/51240 [==============================] - 59s 1ms/step - loss: 1.5719 - accuracy: 0.4089
Epoch 5/200
51240/51240 [==============================] - 61s 1ms/step - loss: 1.5110 - accuracy: 0.4345
Epoch 6/200
51240/51240 [==============================] - 58s 1ms/step - loss: 1.4653 - accuracy: 0.4547
Epoch 7/200
51240/51240 [==============================] - 57s 1ms/step - loss: 1.4126 - accuracy: 0.4731
Epoch 8/200
51240/51240 [==============================] - 57s 1ms/step - loss: 1.3686 - accuracy: 0.4884
Epoch 9/200
51240/51240 [==============================] - 59s 1ms/step - loss: 1.3357 - accuracy: 0.5065
Epoch 10/200
51240/51240 [====================

In [77]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

7320/7320 [==============================] - 3s 420us/step
Test loss: 0.5328557654867607
Test accuracy: 0.8245901465415955
